In [1]:
import cvxpy as cp
import numpy as np

In [2]:
x = cp.Variable(shape=(2,1), name="x")
A = np.array([[4,3],[-3,4]])

constraints = [cp.matmul(A, x) <= 12, x<=2, x>=0]
objective = cp.Maximize(cp.sum(x, axis=0))
problem = cp.Problem(objective, constraints)

solution = problem.solve()
print(solution)

print(x.value)

3.499999999929633
[[1.5]
 [2. ]]
